<a href="https://colab.research.google.com/github/hvarS/CS60075-Team28-Task-1/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d embeddings

--2021-04-10 06:02:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-10 06:02:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-10 06:02:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [83]:
import os
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

In [84]:
FOLDER_PATH = "/content/drive/MyDrive/CS60075-Team28-Task-1"
DATA_FOLDER = os.path.join(FOLDER_PATH,"data/preprocessed")

In [85]:
# import evaluate function
import sys
sys.path.append(FOLDER_PATH)
from eval import evaluate

In [86]:
data = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_single_train_preprocessed.csv"), index_col=0)
data['token'] = data['token'].astype(str)
data['sentence'] = data['sentence'].astype(str)

data.head()

,corpus,sentence,token,complexity
id,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,bible,behold came river seven cattle sleek fat fed m...,river,0.000000
34R0BODSP1ZBN3DVY8J8XSIY551E5C,bible,fellow bondservant brother prophet keep word book,brother,0.000000
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,bible,man lord land said u know honest men leave one...,brother,0.050000
3BFNCI9LYKQN09BHXHH9CLSX5KP738,bible,shimei sixteen son six daughter brother didnt ...,brother,0.150000
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,bible,put brother far,brother,0.263889


In [87]:
data_multi = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_multi_train_preprocessed.csv"), index_col=0)
data_multi['token'] = data_multi['token'].astype(str)
data_multi['sentence'] = data_multi['sentence'].astype(str)

data_multi.head()

,corpus,sentence,token,complexity
id,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,bible,seventh day sabbath yahweh god shall work son ...,seventh day,0.027778
3WGCNLZJKF877FYC1Q6COKNWTDWD11,bible,let man test own work take pride neighbor,own work,0.050000
3UOMW19E6D6WQ5TH2HDD74IVKTP5CB,bible,understanding made heaven loving kindness endu...,loving kindness,0.050000
36JW4WBR06KF9AXMUL4N476OMF8FHD,bible,remember god also spare according greatness lo...,loving kindness,0.050000
3HRWUH63QU2FH9Q8R7MRNFC7JX2N5A,bible,loving kindness better life lip shall praise,loving kindness,0.075000


In [45]:
# # use if model is to be trained on both data together

# data = pd.concat([data, data_multi])

In [46]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

word_to_vec_map = read_glove_vector('embeddings/glove.6B.300d.txt')
print(len(word_to_vec_map)," words loaded!")

400000  words loaded!


In [88]:
def get_embeddings(sentences, tokens):
    token_emb = []
    for s,t in zip(sentences, tokens):
        
        # fill unk by nan
        # calculate mean over non nan embeddings
        # fill unk by the mean embedding of sentence
        # pad 0 vectors till max_len
        
        temp_emb = [ word_to_vec_map[x] if x in word_to_vec_map else np.full((300,), np.nan) for x in t.split() ]
        
        # calculate mean for filling null values <unk>
        temp_sent_emb = [ word_to_vec_map[x] if x in word_to_vec_map else np.full((300,), np.nan) for x in s.split() ]
        mean_emb = np.nanmean(np.array(temp_sent_emb), axis=0)
        
        # single or multi - will be converted to (1,300) 
        temp_emb = np.mean(np.array([ mean_emb if np.isnan(x[0]) else x for x in temp_emb ]), axis=0)

        token_emb.append(temp_emb)

    return np.array(token_emb)

## CNN Regression Testing 

In [ ]:
from sklearn.model_selection import train_test_split             
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences


In [ ]:
sentences_train_list = list(data['sentence'])
complexity_train_list = list(data['complexity'])

sentences_train,sentences_test,y_train,y_test = train_test_split(
                                                sentences_train_list, complexity_train_list,  
                                                test_size=0.25,  
                                                random_state=1000)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1                          

maxlen = 128

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
  vocab_size = len(word_index) + 1  
  embedding_matrix = np.zeros((vocab_size, embedding_dim))

  with open(filepath) as f:
    for line in f:
      word, *vector = line.split()
      if word in word_index:
        idx = word_index[word] 
        embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

  return embedding_matrix

In [ ]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('embeddings/glove.6B.300d.txt ,tokenizer.word_index,  
                                            embedding_dim' )


## **Test Single Word**



In [ ]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/baselines_ankit/single")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

In [ ]:
sentences_train_list = list(data['sentence'])
complexity_train_list = list(data['complexity'])
tokens_train_list = list(data['token'])

vectors = get_embeddings(sentences_train_list, tokens_train_list)
print(vectors.shape)

(9179, 300)


In [ ]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_single_test_preprocessed.csv"), index_col=0)

In [ ]:
test_df['token'] = test_df['token'].astype(str)
test_df['sentence'] = test_df['sentence'].astype(str)
sentences_test_list = list(test_df['sentence'])
test_tokens_list = list(test_df['token'])

testdf_vectors = get_embeddings(sentences_test_list, test_tokens_list)
testdf_vectors.shape

(917, 300)

In [ ]:
# Linear Regression
reg = LinearRegression().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/linear_regression_baseline.csv", index=False, header=False)

# Gradient Boosting
reg = GradientBoostingRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/gradient_boosting_baseline.csv", index=False, header=False)

# AdaBoost
reg = AdaBoostRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/ada_boost_baseline.csv", index=False, header=False)


# SVM regressor
reg = SVR().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/SVM_baseline.csv", index=False, header=False)

# MLP Regressor
reg = MLPRegressor(hidden_layer_sizes=(150)).fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/MLP_baseline.csv", index=False, header=False)

In [ ]:
evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_single_test_labelled_preprocessed.csv")


For file linear_regression_baseline.csv
pearson  :  0.6715029466644609
spearman :  0.6548482413646795
mae      :  0.0759823204469714
mse      :  0.009499099203375246
r2       :  0.4130951539079504

For file gradient_boosting_baseline.csv
pearson  :  0.6939712352149733
spearman :  0.6731328271116146
mae      :  0.0715689474868356
mse      :  0.008668781235018004
r2       :  0.46439661197178606

For file ada_boost_baseline.csv
pearson  :  0.6631336571796327
spearman :  0.6357005159629594
mae      :  0.08355461007871447
mse      :  0.011214615888546787
r2       :  0.30710141339398755

For file SVM_baseline.csv
pearson  :  0.6678429730298119
spearman :  0.6568058055125834
mae      :  0.07496900141336407
mse      :  0.009234749134759981
r2       :  0.42942810643464235

For file MLP_baseline.csv
pearson  :  0.5207687017064077
spearman :  0.5172088187127104
mae      :  0.10987027960820457
mse      :  0.02178758707478297
r2       :  -0.3461529525135425


In [ ]:
For file gradient_boosting_baseline.csv
pearson  :  0.7214444729661804
spearman :  0.6954546622318855
mae      :  0.06718193713274725
mse      :  0.00777120977316972
r2       :  0.5198533483837506

## **Multi Word**


In [ ]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/baselines_ankit/multi")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

In [ ]:
#  DO NOT run if both is being trained together
sentences_train_list = list(data_multi['sentence'])
complexity_train_list = list(data_multi['complexity'])
tokens_train_list = list(data_multi['token'])

In [ ]:
#  DO NOT run if both is being trained together
vectors = get_embeddings(sentences_train_list, tokens_train_list)
print(vectors.shape)

(1517, 300)


In [ ]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, "lcp_multi_test_preprocessed.csv"), index_col=0)

In [ ]:
test_df['token'] = test_df['token'].astype(str)
test_df['sentence'] = test_df['sentence'].astype(str)
sentences_test_list = list(test_df['sentence'])
test_tokens_list = list(test_df['token'])

In [ ]:
testdf_vectors = get_embeddings(sentences_test_list, test_tokens_list)
testdf_vectors.shape

(184, 300)

In [ ]:
# Linear Regression
reg = LinearRegression().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/linear_regression_baseline.csv", index=False, header=False)

# Gradient Boosting
reg = GradientBoostingRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/gradient_boosting_baseline.csv", index=False, header=False)

# AdaBoost
reg = AdaBoostRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/ada_boost_baseline.csv", index=False, header=False)


# SVM regressor
reg = SVR().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/SVM_baseline.csv", index=False, header=False)

# MLP Regressor
reg = MLPRegressor(hidden_layer_sizes=(150)).fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(testdf_vectors)

pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/MLP_baseline.csv", index=False, header=False)

In [ ]:
evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_multi_test_labelled_preprocessed.csv")


For file linear_regression_baseline.csv
pearson  :  0.8179201155507047
spearman :  0.8049744407871943
mae      :  0.10626544460880245
mse      :  0.0162353528148035
r2       :  0.3273702640454511

For file gradient_boosting_baseline.csv
pearson  :  0.8193283866008076
spearman :  0.8090107910611813
mae      :  0.10516079075843644
mse      :  0.01603721066214149
r2       :  0.3355792820659891

For file ada_boost_baseline.csv
pearson  :  0.7941548334757821
spearman :  0.7748241073073495
mae      :  0.09844777180132516
mse      :  0.014233323214324571
r2       :  0.41031423556884616

For file SVM_baseline.csv
pearson  :  0.8137261535536578
spearman :  0.8074492340697415
mae      :  0.07244008552665147
mse      :  0.008700322647294615
r2       :  0.6395461317210687

For file MLP_baseline.csv
pearson  :  0.6233131383622625
spearman :  0.5826967735152306
mae      :  0.1111621001267914
mse      :  0.019383115877151115
r2       :  0.19695862090925698


In [ ]:
For file linear_regression_baseline.csv
pearson  :  0.7384735689252496
spearman :  0.7096751511559939
mae      :  0.08420414291697727
mse      :  0.011177467719053704
r2       :  0.5369181534723191

For file gradient_boosting_baseline.csv
pearson  :  0.7870767227623038
spearman :  0.7664566772179571
mae      :  0.07831874126395687
mse      :  0.009391251928283358
r2       :  0.6109209712372773

For file SVM_baseline.csv
pearson  :  0.7747686594582374
spearman :  0.7563503882622532
mae      :  0.07906457106883466
mse      :  0.009776174954316456
r2       :  0.5949736323456092

## Features - load data

In [109]:
f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_single_train_features.csv"), index_col=0)
f1['token'] = f1['token'].astype(str)
f1['sentence'] = f1['sentence'].astype(str)
f1.set_index("id", inplace=True)

# drop unwanted features
f1.drop(['parse', 'lemma'], axis=1, inplace=True)

print(f1.columns)

f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_single_train_preprocessed.csv"), index_col=0)
f2['token'] = f2['token'].astype(str)
f2['sentence'] = f2['sentence'].astype(str)
print(f2.columns)

features = f1.merge(f2, on=['id','sentence', 'corpus', 'token', 'complexity'])

Index(['sentence', 'corpus', 'token', 'complexity', 'token_length',
       'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms',
       'google frequency'],
      dtype='object')
Index(['corpus', 'sentence', 'token', 'complexity', 'biomedical', 'bible',
       'subtitles', 'wiki', 'familarity'],
      dtype='object')


In [110]:
# fill pos nan by NN, as they are in majority
features['pos'] = features['pos'].fillna('NN')
features['token_length'] = features['token_length'].fillna(0)

# categorical encoding
labels = dict(features['pos'].value_counts())
labels = { k:i for i,k in enumerate(labels)}
labels['POS'] = len(labels)
print(labels)


def get_vowels(word):
    val = 0
    for w in word:
        if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
            val+=1
    return val

features['token_vowels'] = features['token'].apply(lambda x: get_vowels(x) )


features['pos'] = features['pos'].apply((lambda x: labels[x]))

# scaler = preprocessing.StandardScaler()
# features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']] =  \
#     scaler.fit_transform(features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']])


features.head()

{'NN': 0, 'JJ': 1, 'NNP': 2, 'NNS': 3, 'VBG': 4, 'VB': 5, 'RB': 6, 'VBP': 7, 'UH': 8, 'CD': 9, 'VBN': 10, 'FW': 11, 'VBZ': 12, 'IN': 13, 'JJR': 14, 'VBD': 15, 'PRP': 16, 'GW': 17, 'MD': 18, 'JJS': 19, 'WRB': 20, 'AFX': 21, 'LS': 22, 'POS': 23}


,sentence,corpus,token,complexity,token_length,syllables,pos,dep num,synonyms,hypernyms,hyponyms,google frequency,biomedical,bible,subtitles,wiki,familarity,token_vowels
id,,,,,,,,,,,,,,,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,behold came river seven cattle sleek fat fed m...,bible,river,0.000000,5.0,2,0,0,1,1,0,173.485953,1,0,0,0,0,2
34R0BODSP1ZBN3DVY8J8XSIY551E5C,fellow bondservant brother prophet keep word book,bible,brother,0.000000,7.0,2,0,3,5,1,3,112.198857,1,0,0,0,0,2
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,man lord land said u know honest men leave one...,bible,brother,0.050000,7.0,2,0,1,5,1,3,112.198857,1,0,0,0,0,2
3BFNCI9LYKQN09BHXHH9CLSX5KP738,shimei sixteen son six daughter brother didnt ...,bible,brother,0.150000,7.0,2,0,4,5,1,3,112.198857,1,0,0,0,0,2
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,put brother far,bible,brother,0.263889,7.0,2,0,2,5,1,3,112.198857,1,0,0,0,0,2


In [111]:
multi_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_multi_train_split_features.csv"), index_col=0)
multi_f1['token'] = multi_f1['token'].astype(str)
multi_f1['sentence'] = multi_f1['sentence'].astype(str)
multi_f1.set_index("id", inplace=True)

# drop unwanted features
multi_f1.drop(['parse', 'token1', 'token2', 'lemma1', 'lemma2', 'Unnamed: 0.1'], axis=1, inplace=True)

multi_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_multi_train_preprocessed.csv"), index_col=0)
multi_f2['token'] = multi_f2['token'].astype(str)
multi_f2['sentence'] = multi_f2['sentence'].astype(str)

multi_features = multi_f1.merge(multi_f2, on=['id','sentence', 'corpus', 'token', 'complexity'])
multi_features.head(2)

,sentence,corpus,token,complexity,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,seventh day sabbath yahweh god shall work son ...,bible,seventh day,0.027778,11,3,JJ,0,NN,1,4,1,0,10,1,7,24.522564,682.298213,3,1,0,1,0,0.0
3WGCNLZJKF877FYC1Q6COKNWTDWD11,let man test own work take pride neighbor,bible,own work,0.050000,8,2,JJ,0,NN,3,2,0,1,34,1,27,0.000000,1022.711588,2,1,1,1,1,0.0


In [112]:
# fill pos nan by NN, as they are in majority
multi_features['pos2'] = multi_features['pos2'].fillna('NN')

multi_features['pos1'] = multi_features['pos1'].apply((lambda x: labels[x]))
multi_features['pos2'] = multi_features['pos2'].apply((lambda x: labels[x]))

# scaler = preprocessing.StandardScaler()
# multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
#     scaler.fit_transform(multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])


multi_features.head()

,sentence,corpus,token,complexity,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,,
3S37Y8CWI80N8KVM53U4E6JKCDC4WE,seventh day sabbath yahweh god shall work son ...,bible,seventh day,0.027778,11,3,1,0,0,1,4,1,0,10,1,7,24.522564,682.298213,3,1,0,1,0,0.0
3WGCNLZJKF877FYC1Q6COKNWTDWD11,let man test own work take pride neighbor,bible,own work,0.050000,8,2,1,0,0,3,2,0,1,34,1,27,0.000000,1022.711588,2,1,1,1,1,0.0
3UOMW19E6D6WQ5TH2HDD74IVKTP5CB,understanding made heaven loving kindness endu...,bible,loving kindness,0.050000,15,4,4,0,0,2,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,0.0
36JW4WBR06KF9AXMUL4N476OMF8FHD,remember god also spare according greatness lo...,bible,loving kindness,0.050000,15,4,4,1,0,2,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,0.0
3HRWUH63QU2FH9Q8R7MRNFC7JX2N5A,loving kindness better life lip shall praise,bible,loving kindness,0.075000,15,4,4,0,0,0,10,1,12,3,1,4,339.132903,14.221491,4,1,1,1,1,0.0


In [113]:
# merge both single and multi features

features['pos1'] = features['pos'].copy()
features['pos2'] = features['pos'] 

features['dep num1'] = features['dep num'] 
features['dep num2'] = features['dep num'] 

features['synonyms1'] = features['synonyms'] 
features['synonyms2'] = features['synonyms'] 

features['hypernyms1'] = features['hypernyms'] 
features['hypernyms2'] = features['hypernyms'] 

features['hyponyms1'] = features['hyponyms'] 
features['hyponyms2'] = features['hyponyms'] 

features['google frequency1'] = features['google frequency'] 
features['google frequency2'] = features['google frequency1'] 

features.drop(['pos','dep num', 'synonyms', 'hyponyms', 'hypernyms', 'google frequency'], axis=1, inplace=True)

features = features.append( multi_features)
print(len(features))

scaler = preprocessing.StandardScaler()
features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
    scaler.fit_transform(features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])

features.head()

9179


,sentence,corpus,token,complexity,token_length,syllables,biomedical,bible,subtitles,wiki,familarity,token_vowels,pos1,pos2,dep num1,dep num2,synonyms1,synonyms2,hypernyms1,hypernyms2,hyponyms1,hyponyms2,google frequency1,google frequency2
id,,,,,,,,,,,,,,,,,,,,,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,behold came river seven cattle sleek fat fed m...,bible,river,0.000000,-0.797042,-0.505404,1,0,0,0,0.0,-0.689858,-0.323568,-0.248693,-0.707506,-0.831106,-0.729400,-0.754824,0.336733,0.195761,-0.333525,-0.369612,0.419993,0.466245
34R0BODSP1ZBN3DVY8J8XSIY551E5C,fellow bondservant brother prophet keep word book,bible,brother,0.000000,-0.337058,-0.505404,1,0,0,0,0.0,-0.689858,-0.323568,-0.248693,1.709990,1.365895,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,man lord land said u know honest men leave one...,bible,brother,0.050000,-0.337058,-0.505404,1,0,0,0,0.0,-0.689858,-0.323568,-0.248693,0.098326,-0.098772,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3BFNCI9LYKQN09BHXHH9CLSX5KP738,shimei sixteen son six daughter brother didnt ...,bible,brother,0.150000,-0.337058,-0.505404,1,0,0,0,0.0,-0.689858,-0.323568,-0.248693,2.515822,2.098228,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,put brother far,bible,brother,0.263889,-0.337058,-0.505404,1,0,0,0,0.0,-0.689858,-0.323568,-0.248693,0.904158,0.633561,-0.024553,-0.063647,0.336733,0.195761,-0.168281,-0.211593,0.088143,0.097424


## Test with features - single

In [114]:
sentences_train_list = list(features['sentence'])
complexity_train_list = list(features['complexity'])
tokens_train_list = list(features['token'])

vectors = get_embeddings(sentences_train_list, tokens_train_list)
print(vectors.shape)

# f_vectors = features[['token_length', 'token_vowels', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values
# or 

f_vectors = features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

print(f_vectors.shape)
vectors = np.concatenate((vectors, f_vectors), axis=1)

(9179, 300)
(9179, 20)


In [115]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/baselines_ankit/features/single")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

In [116]:
test_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_single_test_features.csv"), index_col=0)
test_f1['token'] = test_f1['token'].astype(str)
test_f1['sentence'] = test_f1['sentence'].astype(str)
test_f1.set_index("id", inplace=True)

# drop unwanted features
test_f1.drop(['parse', 'lemma'], axis=1, inplace=True)

test_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_single_test_preprocessed.csv"), index_col=0)
test_f2['token'] = test_f2['token'].astype(str)
test_f2['sentence'] = test_f2['sentence'].astype(str)

test_features = test_f1.merge(test_f2, on=['id','sentence', 'corpus', 'token'])
test_features.head()

,sentence,corpus,token,token_length,syllables,pos,dep num,synonyms,hypernyms,hyponyms,google frequency,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,
39TX062QX1OHFOH8FUL76K5L7D3X3S,speak much prince world come nothing,bible,prince,6,1,NN,0,1,1,10,57.555460,1,0,0,0,0
3CIS7GGG65JS8I3AZ9RG54AE4MUUEA,house shall turned others field wife together ...,bible,inhabitant,10,4,NN,0,1,1,30,1.907882,1,0,1,0,0
379OL9DBSSESUVWY1Z8JGBFG9BTY92,stranger terrible nation cut left mountain val...,bible,bough,5,1,NN,2,1,1,0,1.513619,1,1,1,1,0
3DFYDSXB2W00JYP2DA272KN69UQUJV,sharpen tongue like sword aim arrow deadly word,bible,arrow,5,2,NN,0,2,1,0,22.707380,0,0,1,0,0
31YWE12TE0CZG7IVH6OXJ1H1CFPX7X,obey leader submit watch behalf soul give acco...,bible,account,7,2,NN,0,14,1,7,216.439551,1,1,1,0,0


In [118]:
# # fill pos nan by NN, as they are in majority
# test_features['pos'] = test_features['pos'].fillna('NN')

# # categorical encoding
# test_features['pos'] = test_features['pos'].apply((lambda x: labels[x]))

# def get_vowels(word):
#     val = 0
#     for w in word:
#         if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
#             val+=1
#     return val

# test_features['token_vowels'] = test_features['token'].apply(lambda x: get_vowels(x) )

# test_features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']] =  \
#     scaler.transform(test_features[['token_length', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'familarity', 'token_vowels']])

# ##########  for all features

# fill pos nan by NN, as they are in majority
test_features['pos'] = test_features['pos'].fillna('NN')
test_features['token_length'] = test_features['token_length'].fillna(0)

test_features['pos'] = test_features['pos'].apply((lambda x: labels[x]))

def get_vowels(word):
    val = 0
    for w in word:
        if(w in ['A', 'a', 'E', 'e', 'I', 'i', 'O', 'o', 'U','u']):
            val+=1
    return val

test_features['token_vowels'] = test_features['token'].apply(lambda x: get_vowels(x) )

test_features['pos1'] = test_features['pos'].copy()
test_features['pos2'] = test_features['pos'] 

test_features['dep num1'] = test_features['dep num'] 
test_features['dep num2'] = test_features['dep num'] 

test_features['synonyms1'] = test_features['synonyms'] 
test_features['synonyms2'] = test_features['synonyms'] 

test_features['hypernyms1'] = test_features['hypernyms'] 
test_features['hypernyms2'] = test_features['hypernyms'] 

test_features['hyponyms1'] = test_features['hyponyms'] 
test_features['hyponyms2'] = test_features['hyponyms'] 

test_features['google frequency1'] = test_features['google frequency'] 
test_features['google frequency2'] = test_features['google frequency'] 

test_features.drop(['pos','dep num', 'synonyms', 'hyponyms', 'hypernyms', 'google frequency'], axis=1, inplace=True)


test_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
    scaler.fit_transform(test_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])

test_features.head()

,sentence,corpus,token,token_length,syllables,biomedical,bible,subtitles,wiki,familarity,token_vowels,pos1,pos2,dep num1,dep num2,synonyms1,synonyms2,hypernyms1,hypernyms2,hyponyms1,hyponyms2,google frequency1,google frequency2
id,,,,,,,,,,,,,,,,,,,,,,,
39TX062QX1OHFOH8FUL76K5L7D3X3S,speak much prince world come nothing,bible,prince,-0.359207,-1.209359,1,0,0,0,0.0,-0.569449,-0.215025,-0.215025,-0.84719,-0.84719,-0.786155,-0.786155,0.23992,0.23992,0.429456,0.429456,-0.266096,-0.266096
3CIS7GGG65JS8I3AZ9RG54AE4MUUEA,house shall turned others field wife together ...,bible,inhabitant,1.207464,1.600562,1,0,1,0,0.0,1.029891,-0.215025,-0.215025,-0.84719,-0.84719,-0.786155,-0.786155,0.23992,0.23992,2.359902,2.359902,-0.593957,-0.593957
379OL9DBSSESUVWY1Z8JGBFG9BTY92,stranger terrible nation cut left mountain val...,bible,bough,-0.750874,-1.209359,1,1,1,1,0.0,-0.569449,-0.215025,-0.215025,0.67311,0.67311,-0.786155,-0.786155,0.23992,0.23992,-0.535767,-0.535767,-0.596279,-0.596279
3DFYDSXB2W00JYP2DA272KN69UQUJV,sharpen tongue like sword aim arrow deadly word,bible,arrow,-0.750874,-0.272719,0,0,1,0,0.0,-0.569449,-0.215025,-0.215025,-0.84719,-0.84719,-0.597733,-0.597733,0.23992,0.23992,-0.535767,-0.535767,-0.471412,-0.471412
31YWE12TE0CZG7IVH6OXJ1H1CFPX7X,obey leader submit watch behalf soul give acco...,bible,account,0.032461,-0.272719,1,1,1,0,0.0,0.230221,-0.215025,-0.215025,-0.84719,-0.84719,1.663337,1.663337,0.23992,0.23992,0.139889,0.139889,0.670006,0.670006


In [119]:
sentences_test_list = list(test_features['sentence'])
test_tokens_list = list(test_features['token'])

test_vectors = get_embeddings(sentences_test_list, test_tokens_list)
print(test_vectors.shape)

# test_f_vectors = test_features[['token_length', 'token_vowels', 'syllables', 'pos', 'dep num', 'synonyms', 'hypernyms', 'hyponyms', 'google frequency', 'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

test_f_vectors = test_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values

test_vectors = np.concatenate((test_vectors, test_f_vectors), axis=1)
print(test_vectors.shape)

(917, 300)
(917, 320)


In [120]:
# Linear Regression
reg = LinearRegression().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/linear_regression_baseline.csv", index=False, header=False)

# Gradient Boosting
reg = GradientBoostingRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/gradient_boosting_baseline.csv", index=False, header=False)

# AdaBoost
reg = AdaBoostRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/ada_boost_baseline.csv", index=False, header=False)


# SVM regressor
reg = SVR().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/SVM_baseline.csv", index=False, header=False)

# MLP Regressor
reg = MLPRegressor(hidden_layer_sizes=(150)).fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/MLP_baseline.csv", index=False, header=False)

In [121]:
evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_single_test_labelled_preprocessed.csv")


For file linear_regression_baseline.csv
pearson  :  0.6764732930844866
spearman :  0.6760017156397601
mae      :  0.07569338580364902
mse      :  0.009511516322174056
r2       :  0.4123279582990307

For file gradient_boosting_baseline.csv
pearson  :  0.7015786271547141
spearman :  0.672954947440128
mae      :  0.07348020617004901
mse      :  0.009052928685933177
r2       :  0.4406619403225088

For file ada_boost_baseline.csv
pearson  :  0.6829921537603776
spearman :  0.6725387317487636
mae      :  0.0788341859096584
mse      :  0.010327774663127768
r2       :  0.3618951787573066

For file SVM_baseline.csv
pearson  :  0.6968593439567711
spearman :  0.6843007666696466
mae      :  0.07361390870122693
mse      :  0.008702552238726511
r2       :  0.46231005983567686

For file MLP_baseline.csv
pearson  :  0.5446037367508759
spearman :  0.526313892693398
mae      :  0.1081767212887482
mse      :  0.01973212019436534
r2       :  -0.2191552817585971


## Test with features - multi

In [122]:
sentences_train_list = list(multi_features['sentence'])
complexity_train_list = list(multi_features['complexity'])
tokens_train_list = list(multi_features['token'])

vectors = get_embeddings(sentences_train_list, tokens_train_list)
print(vectors.shape)

f_vectors = multi_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values
print(f_vectors.shape)
vectors = np.concatenate((vectors, f_vectors), axis=1)

(1517, 300)
(1517, 20)


In [123]:
SUBMISSION_FOLDER = os.path.join(FOLDER_PATH,"predictions/baselines_ankit/features/multi")

if( not os.path.exists(SUBMISSION_FOLDER)):
    os.makedirs(SUBMISSION_FOLDER)

In [124]:
test_multi_f1 = pd.read_csv(os.path.join(FOLDER_PATH, "data/extra_features/lcp_multi_test_split_features.csv"), index_col=0)
test_multi_f1['token'] = test_multi_f1['token'].astype(str)
test_multi_f1['sentence'] = test_multi_f1['sentence'].astype(str)
test_multi_f1.set_index("id", inplace=True)

# drop unwanted features
test_multi_f1.drop(['parse', 'token1', 'token2', 'lemma1', 'lemma2', 'Unnamed: 0.1'], axis=1, inplace=True)

test_multi_f2 = pd.read_csv(os.path.join(FOLDER_PATH, "data/added_corpus_presence/lcp_multi_test_preprocessed.csv"), index_col=0)
test_multi_f2['token'] = test_multi_f2['token'].astype(str)
test_multi_f2['sentence'] = test_multi_f2['sentence'].astype(str)

test_multi_features = test_multi_f1.merge(test_multi_f2, on=['id','sentence', 'corpus', 'token'])
test_multi_features['token'] = test_multi_f2['token'].astype(str)
test_multi_features.head(2)

,sentence,corpus,token,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,
3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,come intending bring bound chief priest,bible,chief priest,12,2,JJ,0,NN,2,4,1,10,2,2,11,0.000000,39.121551,4,1,0,1,0,0.0
302U8RURJZ1WF35NXY44RD66WL4NVH,day lord take away beauty anklet headband cres...,bible,crescent necklace,17,4,NN,1,NN,2,2,1,0,1,1,3,4.830131,4.021996,5,1,1,1,1,0.0


In [125]:
# fill pos nan by NN, as they are in majority
test_multi_features['pos2'] = test_multi_features['pos2'].fillna('NN')

test_multi_features['pos1'] = test_multi_features['pos1'].apply((lambda x: labels[x]))
test_multi_features['pos2'] = test_multi_features['pos2'].apply((lambda x: labels[x]))

test_multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']] =  \
    scaler.fit_transform(test_multi_features[['token_length', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 'google frequency1', 'google frequency2', 'familarity', 'token_vowels']])


test_multi_features.head()

,sentence,corpus,token,token_length,syllables,pos1,dep num1,pos2,dep num2,synonyms1,hypernyms1,hyponyms1,synonyms2,hypernyms2,hyponyms2,google frequency1,google frequency2,token_vowels,biomedical,bible,subtitles,wiki,familarity
id,,,,,,,,,,,,,,,,,,,,,,,
3A9LA2FRWSEW9WO7UFA9AE6VQK3XHL,come intending bring bound chief priest,bible,chief priest,-0.901199,-1.561702,-0.007544,-0.303363,-0.179239,0.195616,-0.150640,0.803614,1.337977,-0.630931,3.91578,0.010441,-0.368197,-0.306098,-0.946411,1,0,1,0,0.0
302U8RURJZ1WF35NXY44RD66WL4NVH,day lord take away beauty anklet headband cres...,bible,crescent necklace,0.281544,-0.569118,-0.701592,1.291460,-0.179239,0.195616,-0.564339,0.803614,-0.377617,-0.814620,0.00000,-0.212299,-0.354034,-0.554398,-0.444568,1,1,1,1,0.0
3UDTAB6HH6ZVX00DTRXAOJLWX0B094,unclean shall take ash burning sin offering ru...,bible,sin offering,-0.901199,-0.569118,-0.701592,4.481106,6.102647,-0.470928,0.676757,0.803614,-0.206058,1.940711,0.00000,-0.295826,-0.216363,0.229547,-0.946411,1,0,1,0,0.0
3L2OEKSTW9ASGQDOW725GFK5P77Y8D,precious treasure oil dwelling wise foolish ma...,bible,precious treasure,0.281544,-0.569118,-0.007544,-0.303363,-0.179239,-0.470928,0.056209,-1.141978,-0.377617,0.103824,0.00000,-0.184456,-0.300091,-0.479155,1.060962,1,0,1,1,0.0
39N6W9XWRDN795J6F5ET8S13DQKYGT,long god shall adversary reproach,bible,adversary reproach,0.518093,0.423467,-0.701592,-0.303363,-0.179239,1.528705,-0.771188,0.803614,0.308621,-0.447242,0.00000,-0.240141,-0.351906,-0.547828,0.057276,1,0,1,1,0.0


In [126]:
sentences_test_list = list(test_multi_features['sentence'])
test_tokens_list = list(test_multi_features['token'])

test_vectors = get_embeddings(sentences_test_list, test_tokens_list)
print(test_vectors.shape)

test_f_vectors = test_multi_features[['token_length', 'token_vowels', 'syllables', 'pos1', 'pos2', 'dep num1', 'dep num2', 
                        'synonyms1', 'synonyms2', 'hypernyms1', 'hypernyms2', 'hyponyms1', 'hyponyms2', 
                        'google frequency1', 'google frequency2', 
                        'biomedical', 'bible', 'subtitles', 'wiki', 'familarity']].values
test_vectors = np.concatenate((test_vectors, test_f_vectors), axis=1)
print(test_vectors.shape)

(184, 300)
(184, 320)


In [127]:
# Linear Regression
reg = LinearRegression().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/linear_regression_baseline.csv", index=False, header=False)

# Gradient Boosting
reg = GradientBoostingRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/gradient_boosting_baseline.csv", index=False, header=False)

# AdaBoost
reg = AdaBoostRegressor().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/ada_boost_baseline.csv", index=False, header=False)


# SVM regressor
reg = SVR().fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/SVM_baseline.csv", index=False, header=False)

# MLP Regressor
reg = MLPRegressor(hidden_layer_sizes=(150)).fit(vectors, np.array(complexity_train_list))
y_pred = reg.predict(test_vectors)

pred = pd.DataFrame({"ID":test_multi_features.index, "complexity":y_pred})
pred.to_csv(SUBMISSION_FOLDER+"/MLP_baseline.csv", index=False, header=False)

In [128]:
evaluate(SUBMISSION_FOLDER, FOLDER_PATH+"/references/lcp_multi_test_labelled_preprocessed.csv")


For file linear_regression_baseline.csv
pearson  :  0.73181025788052
spearman :  0.706710151394026
mae      :  0.08622353323462664
mse      :  0.01143825876694363
r2       :  0.5261135953156546

For file gradient_boosting_baseline.csv
pearson  :  0.7919895516580935
spearman :  0.7663306747668398
mae      :  0.07848888431095036
mse      :  0.009499547922764287
r2       :  0.606434274397143

For file ada_boost_baseline.csv
pearson  :  0.8032811552045976
spearman :  0.7729281948209605
mae      :  0.0811253219565396
mse      :  0.009900449092772618
r2       :  0.589824961916992

For file SVM_baseline.csv
pearson  :  0.6801334456449308
spearman :  0.6922099218449158
mae      :  0.11964508647478064
mse      :  0.022019511294338164
r2       :  0.0877329099830747

For file MLP_baseline.csv
pearson  :  0.40296635924446544
spearman :  0.4067828297544516
mae      :  0.19860332251870674
mse      :  0.06018139513600453
r2       :  -1.49331174883969
